In [1]:
import os

import numpy as np


# Saving because I don't want to red
save_dir = "data/CamVid/np_saves/256_256"

X_train = np.load(save_dir + "/x_train.npy")
Y_train = np.load(save_dir+"/y_train.npy")


In [2]:
import numpy as np
from classification_models.tfkeras import Classifiers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape

    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output



def DeeplabV3Plus(image_size, num_classes):

    model_input = keras.Input(shape=(image_size[0], image_size[1], 3))
    
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    resnet18 = ResNet18(weights='imagenet',include_top=False, input_tensor=model_input)
    
    x = resnet18.get_layer("relu1").output
    x = DilatedSpatialPyramidPooling(x)


    input_a = layers.UpSampling2D(
        size=(image_size[0] // 4 // x.shape[1], image_size[1] // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    
    input_b = resnet18.get_layer("stage2_unit1_relu1").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)
    
    input_a = tf.keras.layers.CenterCrop(input_b.shape[1], input_b.shape[2])(input_a)
    
    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size[0] // x.shape[1], image_size[1] // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same",activation="softmax")(x)
    
    model = keras.Model(inputs=model_input, outputs=model_output)
    
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    
    model.compile(
                 optimizer=keras.optimizers.Adam(learning_rate=0.001),
                 loss=loss,
                 metrics=["accuracy"])
    return model


In [5]:
model = DeeplabV3Plus((256, 256), 12)

In [6]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True
)

history = model.fit(
    X_train, Y_train, epochs=100, verbose=1, batch_size=4,  validation_split=0.2, callbacks=[early_stopping_cb]
)


Epoch 1/100


2022-03-23 16:16:31.604971: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-03-23 16:16:34.162070: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-03-23 16:16:34.162638: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-03-23 16:16:34.162662: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-03-23 16:16:34.163025: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-03-23 16:16:34.163078: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


112/112 [==============================] - 22s 83ms/step - loss: 0.6830 - accuracy: 0.7765 - val_loss: 2.0581 - val_accuracy: 0.3448
Epoch 2/100
112/112 [==============================] - 9s 78ms/step - loss: 0.4838 - accuracy: 0.8329 - val_loss: 4.9828 - val_accuracy: 0.3109
Epoch 3/100
112/112 [==============================] - 9s 78ms/step - loss: 0.4036 - accuracy: 0.8604 - val_loss: 6.3558 - val_accuracy: 0.1752
Epoch 4/100
112/112 [==============================] - 9s 78ms/step - loss: 0.3606 - accuracy: 0.8747 - val_loss: 4.3947 - val_accuracy: 0.3365
Epoch 5/100
112/112 [==============================] - 9s 78ms/step - loss: 0.3206 - accuracy: 0.8876 - val_loss: 3.1979 - val_accuracy: 0.3708
Epoch 6/100
112/112 [==============================] - 9s 78ms/step - loss: 0.3014 - accuracy: 0.8942 - val_loss: 3.0685 - val_accuracy: 0.5067
Epoch 7/100
112/112 [==============================] - 9s 78ms/step - loss: 0.2853 - accuracy: 0.8988 - val_loss: 1.3000 - val_accuracy: 0.7031
Epo

In [7]:
model.save("saved_models/256_256_model")

2022-03-23 16:21:41.656620: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_models/256_256_model/assets


/home/vuran/prashant/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [8]:
import json
with open("histories/256_256_model.json", "w") as f:
    json.dump(history.history, f)